In [1]:
#导包
import cv2 as cv
import numpy as np

In [2]:
Window_Title = 'Plate Lacate'
plate_file_path = 'images/A82806.jpg'

In [3]:
#1. 装载图片
# origin_image = cv.imread(plate_file_path)
origin_image = cv.imdecode(np.fromfile(plate_file_path, dtype=np.uint8), -1)

In [4]:
cv.imshow('origin_image', origin_image)
cv.waitKey()
cv.destroyAllWindows()

32

In [5]:
#2. 完成高斯模糊-预处理
blured_image = cv.GaussianBlur(origin_image, (5, 5), 0)

In [6]:
cv.imshow('blured_image', blured_image)
cv.waitKey()
cv.destroyAllWindows()

32

In [7]:
#3. 灰度+二值化
gray_image = cv.cvtColor(blured_image, cv.COLOR_BGR2GRAY)
is_success, binary_image = cv.threshold(gray_image, 0, 255, cv.THRESH_OTSU)

In [8]:
cv.imshow('binary_image', binary_image)
cv.waitKey()
cv.destroyAllWindows()

32

In [9]:
#4. 在原图上获取轮廓并绘制
# 获取所有的轮廓，轮廓边框使用最小模式（1个像素）
contours, _ = cv.findContours(binary_image, cv.RETR_CCOMP, cv.CHAIN_APPROX_NONE)
# 在原始图片上用红色线绘制所有轮廓
cv.drawContours(origin_image, contours, -1, (0, 0, 255))

array([[[ 39,  70,  71],
        [ 46,  80,  80],
        [ 55,  92,  90],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[ 42,  72,  73],
        [ 48,  80,  79],
        [ 49,  83,  82],
        ...,
        [123, 129, 142],
        [136, 139, 153],
        [  0,   0, 255]],

       [[ 46,  74,  74],
        [ 36,  67,  64],
        [ 36,  69,  65],
        ...,
        [135, 142, 151],
        [146, 151, 160],
        [  0,   0, 255]],

       ...,

       [[  0,   0, 255],
        [215, 237, 242],
        [218, 243, 247],
        ...,
        [185, 198, 200],
        [182, 195, 197],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [208, 232, 238],
        [212, 236, 242],
        ...,
        [181, 194, 196],
        [180, 193, 195],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]]

In [10]:
# contours

In [11]:
cv.imshow('contours_image', origin_image)
cv.waitKey()
cv.destroyAllWindows()

32

In [12]:
#5. 基于面积（长宽比）获取车牌区域
# 声明所有候选|满足条件的区域列表
candidate_regions = []
# 遍历所有的轮廓
for i in np.arange(len(contours)):
    # 提取所有轮廓的左上坐标，及宽、高
    x, y, w, h = cv.boundingRect(contours[i])
    # 计算宽高比
    ratio = w * 1.0 / h
    # 如果是竖排情况，处理长宽比=取反
    if ratio < 1:
        ratio = 1.0 / ratio
    # 求出该区域面积
    area = w * h
    # 关键条件：宽高比在[2, 5]之间
    # 次要条件：区域面积在[136*36, 136*36*10]之间，主要目的是为了过滤过小、过大的封闭区域
    if area > 136*36 and area < 136*36*10 and ratio > 2.0 and ratio < 5.0:
        # 将满足条件的区域的追加到 候选|满足条件 区域列表中
        # 注意：使用切片方式采样原始图片矩阵，
        # 此时上面的 y（其实是高，对应原始图片的第一个维度）
        # 此时上面的 x（其实是宽，对应原始图片的第二个维度）
        candidate_regions.append(origin_image[y:y+h, x:x+w])
# 如果候选区域没有数据，说明提取车牌区域失败|没有车牌区域
if len(candidate_regions) == 0:
    print('没有找到车牌区域')

In [13]:
#6. 逐个显示提取的车牌候选区域(s)
for i in np.arange(len(candidate_regions)):
    # candidate_regions[i] 中保存的是满足条件的一个车牌区域在原始图片中的采样结果
    cv.imshow(str(i), candidate_regions[i])
    cv.waitKey()
cv.destroyAllWindows()

32

32